In [ ]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# K-Means Clustering



The animation below shows how the algorithm initializes and updates until convergence.

![](kmeans_animation.gif)

Notice, that if we change the initial centroids, it can result in different clusters.

![](kmeans_animation_2.gif)